In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import nltk
import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model


In [ ]:
%%writefile contents.json

{
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi there",
                "How are you",
                "Is anyone there?",
                "Hey",
                "Hola",
                "Hello",
                "Good day",
                "Namaste",
                "yo"
            ],
            "responses": [
                "Hello",
                "Good to see you again",
                "Hi there, how can I help?"
            ],
            "context": [""]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "See you later",
                "Goodbye",
                "Get lost",
                "Till next time",
                "bbye"
            ],
            "responses": [
                "See you!",
                "Have a nice day",
                "Bye! Come back again soon."
            ],
            "context": [""]
        },
        {
            "tag": "about",
            "patterns": [
                "what is pcos",
                "define pcos",
                "tell me about pcos",
                "explain pcos",
                "pcos",
                "describe pcos"
            ],
            "responses": [
                "A hormonal disorder causing enlarged ovaries with small cysts on the outer edges.",
                "Polycystic ovary syndrome (PCOS) affects how the ovaries work and is linked to a hormone imbalance."
            ],
            "context": [""]
        },
        {
            "tag": "medications",
            "patterns": [
                "What medications are available for PCOS?",
                "what drugs to use"
            ],
            "responses": [
                "Clomiphene, Oral contraceptives, glucose-lowering medication, Anti-androgens"
            ],
            "context": [""]
        },
        {
            "tag": "food consumed",
            "patterns": [
                "what food should be consumed",
                "what are the nutrient requirements"
            ],
            "responses": [
                "Whole grains, Lean proteins, Low-fat dairy, fruits and vegetables, vitamins and minerals"
            ],
            "context": [""]
        },
        {
            "tag": "exercises",
            "patterns": [
                "What exercises are recommended for PCOS?",
                "Which exercises are advised for individuals with Polycystic Ovary Syndrome?",
                "What exercises can help manage the symptoms of PCOS?"
            ],
            "responses": [
                "Cardio exercises like running, aerobics, strength training, yoga, muscle building, breathing exercises, High-intensity interval training"
            ],
            "context": [""]
        },
        {
            "tag": "symptoms",
            "patterns": [
                "what are the symptoms",
                "what are the signs",
                "common indicators",
                "explain the symptoms",
                "tell about the symptoms"
            ],
            "responses": [
                "Irregular or missed periods, Excessive hair growth, Acne, infertility",
                "Irregular menstrual cycles or no periods at all, difficulties in conception, unwanted hair growth, weight gain"
            ],
            "context": [""]
        },
        {
            "tag": "causes",
            "patterns": [
                "what causes Polycystic Ovary Syndrome?",
                "Can you explain the cause of PCOS?",
                "what is the reason of pcos"
            ],
            "responses": [
                "The exact cause of PCOS is unknown, but it is believed to involve a combination of genetic and environmental factors."
            ],
            "context": [""]
        },
        {
            "tag": "cure",
            "patterns": [
                "what is the cure of pcos",
                "how to cure pcos?",
                "What are the current treatments or cures for PCOS?",
                "Can PCOS be cured?"
            ],
            "responses": [
                "Currently, there is no cure for PCOS, but symptoms can be managed with lifestyle changes and medications."
            ],
            "context": [""]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "That's helpful",
                "Awesome, thanks",
                "Thanks for helping me"
            ],
            "responses": [
                "Happy to help!",
                "Any time!",
                "My pleasure"
            ],
            "context": [""]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": [
                "Sorry, can't understand you",
                "Please give me more info",
                "Not sure I understand"
            ],
            "context": [""]
        },
        {
            "tag": "age",
            "patterns": [
                "how old are you",
                "when were you made",
                "what is your age"
            ],
            "responses": [
                "I was made in 2020, if that's what you are asking!"
            ],
            "context": [""]
        }
    ]
}




Overwriting contents.json


In [ ]:
with open('contents.json') as content:
    data1 = json.load(content)

In [ ]:
tags=[]
imputs=[]
responses={}
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        imputs.append(lines)
        tags.append(intent['tag'])


In [ ]:
data=pd.DataFrame({'inputs':imputs,'tags':tags})

In [ ]:
data

,inputs,tags
0,Hi there,greeting
1,How are you,greeting
2,Is anyone there?,greeting
3,Hey,greeting
4,Hola,greeting
5,Hello,greeting
6,Good day,greeting
7,Namaste,greeting
8,yo,greeting
9,Bye,goodbye


In [ ]:
import string
data['inputs']=data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs']=data['inputs'].apply(lambda wrd:''.join(wrd))
data

,inputs,tags
0,hi there,greeting
1,how are you,greeting
2,is anyone there,greeting
3,hey,greeting
4,hola,greeting
5,hello,greeting
6,good day,greeting
7,namaste,greeting
8,yo,greeting
9,bye,goodbye


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

10


In [ ]:
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = le.classes_.shape[0]
print("output length : ",output_length)

number of unique words :  83
output length :  11


In [ ]:
from ast import mod
i=Input(shape=(input_shape,))
x=Embedding(vocabulary+1,10)(i)
x=LSTM(10,return_sequences=True)(x)
x=Flatten()(x)
x=Dense(output_length,activation="softmax")(x)
model = Model(i,x)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
train=model.fit(x_train,y_train,epochs=500)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0382 - loss: 2.3979
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0625 - loss: 2.3943
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1528 - loss: 2.3904
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1979 - loss: 2.3860
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1875 - loss: 2.3821
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1771 - loss: 2.3784
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1771 - loss: 2.3758
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1979 - loss: 2.3635
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1875 - loss: 2.3579
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1979 - loss: 2.3517
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1771 - loss: 2.3441
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1771 - lo

In [ ]:
import random
while True:
    texts_p=[]
    prediction_input=input('You: ')
    prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input=''.join(prediction_input)
    texts_p.append(prediction_input)
    prediction_input=tokenizer.texts_to_sequences(texts_p)
    prediction_input=np.array(prediction_input).reshape(-1)
    prediction_input=pad_sequences([prediction_input],input_shape)
    output=model.predict(prediction_input)
    output=output.argmax()
    response_tag=le.inverse_transform([output])[0]
    print("Cyster: ",random.choice(responses[response_tag]))
    if response_tag=="goodbye":
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Cyster:  A hormonal disorder causing enlarged ovaries with small cysts on the outer edges.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Cyster:  The exact cause of PCOS is unknown, but it is believed to involve a combination of genetic and environmental factors.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Cyster:  Polycystic ovary syndrome (PCOS) affects how the ovaries work and is linked to a hormone imbalance.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Cyster:  The exact cause of PCOS is unknown, but it is believed to involve a combination of genetic and environmental factors.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Cyster:  A hormonal disorder causing enlarged ovaries with small cysts on the outer edges.
